### User:

Output the full code for the image annotation tool that 
has been generated throughout this conversation

### Chat GPT:

Certainly! 
Here's the complete code for the image annotation tool 
based on the discussions we've had:

In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import ImageTk, Image, ImageGrab

class Image_Markup(object):
    running_instance = None
    ZOOM_STEP = 0.1
    filepath = ""

    def __init__(self):
        self.root = tk.Tk()
        self.root.title('SEE Image Markup Tool')

        self.zoom_factor = 1.0
        self.zoom = 1.0
        self.pan_start_x = 0
        self.pan_start_y = 0
        self.original_image = None

        self.black_brush_button = tk.Button(self.root, text='Black Brush', command=self.use_black_brush, fg='black')
        self.black_brush_button.grid(row=0, column=1)

        self.white_brush_button = tk.Button(self.root, text='White Brush', command=self.use_white_brush, fg='black')
        self.white_brush_button.grid(row=0, column=2)

        self.browse_button = tk.Button(self.root, text='Browse', command=self.open_image)
        self.browse_button.grid(row=0, column=4)

        self.clear_button = tk.Button(self.root, text='Clear All', command=self.clear_all)
        self.clear_button.grid(row=0, column=5)

        self.choose_size_button = tk.Scale(self.root, from_=1, to=100, orient=tk.HORIZONTAL)
        self.choose_size_button.grid(row=0, column=6)

        self.save_button = tk.Button(self.root, text='Save', command=self.save_canvas)
        self.save_button.grid(row=0, column=9)

        self.zoom_in_button = tk.Button(self.root, text='+', command=self.zoom_in)
        self.zoom_in_button.grid(row=0, column=7)

        self.zoom_out_button = tk.Button(self.root, text='-', command=self.zoom_out)
        self.zoom_out_button.grid(row=0, column=8)

        self.pan_button = tk.Button(self.root, text='Pan', command=self.activate_pan)
        self.pan_button.grid(row=0, column=9)

        self.c_frame = tk.Frame(self.root)
        self.c_frame.grid(row=1, columnspan=11)

        self.c = tk.Canvas(self.c_frame, bg='white')
        self.c.grid(row=0, column=0, sticky='nsew')
        self.c.bind('<Configure>', lambda event: self.root.after_idle(self.update_canvas))

        self.setup()

        self.annotation_canvas = tk.Canvas(self.root)
        self.annotation_canvas.grid(row=0, column=0, sticky='nsew')

        self.c.bind("<ButtonPress-1>", self.start_pan)
        self.c.bind("<B1-Motion>", self.pan)
        self.c.bind("<ButtonRelease-1>", self.end_pan)

        self.c.bind("<MouseWheel>", self.zoom)
        self.c.bind("<Button-4>", self.zoom)
        self.c.bind("<Button-5>", self.zoom)

        self.root.mainloop()

    def setup(self):
        self.old_x = None
        self.old_y = None
        self.line_width = self.choose_size_button.get()
        self.active_button = self.black_brush_button
        self.zoom_in_factor = 1.1
        self.zoom_out_factor = 0.9

    def use_black_brush(self):
        self.brush_color = 'black'
        self.activate_button(self.black_brush_button)

    def use_white_brush(self):
        self.brush_color = 'white'
        self.activate_button(self.white_brush_button)

    def clear_all(self):
        self.c.delete("all")

    def activate_button(self, some_button):
        some_button.config(relief=tk.SUNKEN)
        self.active_button = some_button

    def paint(self, event=None):
        self.line_width = self.choose_size_button.get()
        paint_color = self.brush_color
        if self.old_x and self.old_y:
            self.c.create_line(self.old_x, self.old_y, event.x, event.y,
                               width=self.line_width, fill=paint_color,
                               capstyle=tk.ROUND, smooth=True, splinesteps=36)
        self.old_x = event.x
        self.old_y = event.y

    def reset(self, event=None):
        self.old_x, self.old_y = None, None

    def save_canvas(self):
        x = self.root.winfo_rootx() + self.c.winfo_x()
        y = self.root.winfo_rooty() + self.c.winfo_y()
        x1 = x + self.c.winfo_width()
        y1 = y + self.c.winfo_height()
        print('x, y, x1, y1:', x, y, x1, y1)

        # Fix so that the user is prompted to select the file name and location
        ImageGrab.grab().crop(self.c.bbox()).save("/Users/shadowrainbow/Downloads/edit_IMG_20170623_140853.jpg")

    def zoom_in(self):
        self.zoom_factor += self.ZOOM_STEP
        self.update_canvas()

    def zoom_out(self):
        if self.zoom_factor > self.ZOOM_STEP:
            self.zoom_factor -= self.ZOOM_STEP
            self.update_canvas()

    def update_canvas_frame(self, event):
        self.c.configure(scrollregion=self.c.bbox('all'))

    def activate_pan(self):
        self.c.bind('<ButtonPress-1>', self.start_pan)
        self.c.bind('<B1-Motion>', self.pan)
        self.c.bind('<ButtonRelease-1>', self.end_pan)

    def start_pan(self, event):
        self.pan_start_x = event.x
        self.pan_start_y = event.y

    def pan(self, event):
        dx = event.x - self.pan_start_x
        dy = event.y - self.pan_start_y
        self.c.scan_dragto(dx, dy, gain=1)

    def end_pan(self, event):
        self.c.unbind('<ButtonPress-1>')
        self.c.unbind('<B1-Motion>')
        self.c.unbind('<ButtonRelease-1>')

    def open_image(self):
        filetypes = (("Image files", ("*.png", "*.jpg", "*.jpeg", "*.gif", "*.bmp", "*.tif", "*.tiff")), ("All files", "*.*"))
        filepath = filedialog.askopenfilename(title="Select Image", filetypes=filetypes)

        if filepath:
            try:
                self.original_image = Image.open(filepath)
                self.update_canvas()  # Update the canvas with the loaded image
            except Exception as e:
                messagebox.showerror("Error", f"Failed to open image: {e}")

    def update_canvas(self, event=None):
        if self.original_image is None:
            return

        width = int(self.original_image.width * self.zoom_factor)
        height = int(self.original_image.height * self.zoom_factor)
        self.displayed_image = self.original_image.resize((width, height))

        self.c.config(width=width, height=height)
        self.c.delete('all')

        if self.displayed_image:
            self.img_tk = ImageTk.PhotoImage(self.displayed_image)
            self.c.create_image(0, 0, anchor=tk.NW, image=self.img_tk)

    def zoom(self, event):
        if event.delta > 0:
            self.zoom_in()
        else:
            self.zoom_out()

    @classmethod
    def run(cls):
        if cls.running_instance is None:
            cls.running_instance = cls()
        else:
            cls.running_instance.root.deiconify()


if __name__ == '__main__':
    Image_Markup.run()

2023-06-26 17:58:44.463 python[29708:3773787] +[CATransaction synchronize] called within transaction
Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/shadowrainbow/anaconda3/lib/python3.10/tkinter/__init__.py", line 1921, in __call__
    return self.func(*args)
TypeError: 'float' object is not callable
Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/shadowrainbow/anaconda3/lib/python3.10/tkinter/__init__.py", line 1921, in __call__
    return self.func(*args)
TypeError: 'float' object is not callable
Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/shadowrainbow/anaconda3/lib/python3.10/tkinter/__init__.py", line 1921, in __call__
    return self.func(*args)
TypeError: 'float' object is not callable
Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/shadowrainbow/anaconda3/lib/python3.10/tkinter/__init__.py", line 1921, in __call__
    return self.func(*args)
TypeE